In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from dataset import Downstream_Dataset, DataAugmentation, LoadPretrainData
import torch
import torch.nn as nn
from torchmetrics import R2Score
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader

/home/mm22d016/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fingerprint = torch.empty(368, 768)
pred_output = torch.empty(368,1)

In [3]:
class DownstreamRegression(nn.Module):
    def __init__(self, drop_rate=0.1):
        super(DownstreamRegression, self).__init__()
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))
        
        self.Regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.PretrainedModel.config.hidden_size, self.PretrainedModel.config.hidden_size),
            nn.SiLU(),
            nn.Linear(self.PretrainedModel.config.hidden_size, 1)
        )

    def forward(self, input_ids, attention_mask,step):
        outputs = self.PretrainedModel(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state[:, 0, :] #fingerprint
        fingerprint[step] = logits
        output = self.Regressor(logits)
        return output
    
def test(model, loss_fn, train_dataloader,device):

    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(train_dataloader):
            print(f'Smiles: {step+1}')
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            prop = batch["prop"].to(device).float()
            outputs = model(input_ids, attention_mask,step).float()
            pred_output[step] = outputs

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = pd.read_csv('data/Eea.csv')
original_output = train_data['value'].tolist()

In [5]:
saved_state = torch.load('ckpt/Eea/Eea_best_model.pt')
vocab_sup = pd.read_csv('data/vocab/vocab_sup_PE_I.csv', header=None).values.flatten().tolist()

scaler = StandardScaler()
train_data.iloc[:, 1] = scaler.fit_transform(train_data.iloc[:, 1].values.reshape(-1, 1))


PretrainedModel = RobertaModel.from_pretrained('ckpt/pretrain.pt')
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
tokenizer.add_tokens(vocab_sup)
train_dataset = Downstream_Dataset(train_data, tokenizer, 411)

model = DownstreamRegression(drop_rate=0.1).to(device)
model = model.double()
model.load_state_dict(saved_state['model'])
loss_fn = nn.MSELoss()

train_dataloader = DataLoader(train_dataset, 1, shuffle=False, num_workers=8)

Some weights of the model checkpoint at ckpt/pretrain.pt were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ckpt/pretrain.pt and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you 

In [6]:
print('Hello')
test(model, loss_fn, train_dataloader, device)

fingerprint = fingerprint.detach().cpu().numpy().tolist()
pred_output = pred_output.detach().cpu().numpy().tolist()

Hello
Smiles: 1
Smiles: 2
Smiles: 3
Smiles: 4
Smiles: 5
Smiles: 6
Smiles: 7
Smiles: 8
Smiles: 9
Smiles: 10
Smiles: 11
Smiles: 12
Smiles: 13
Smiles: 14
Smiles: 15
Smiles: 16
Smiles: 17
Smiles: 18
Smiles: 19
Smiles: 20
Smiles: 21
Smiles: 22
Smiles: 23
Smiles: 24
Smiles: 25
Smiles: 26
Smiles: 27
Smiles: 28
Smiles: 29
Smiles: 30
Smiles: 31
Smiles: 32
Smiles: 33
Smiles: 34
Smiles: 35
Smiles: 36
Smiles: 37
Smiles: 38
Smiles: 39
Smiles: 40
Smiles: 41
Smiles: 42
Smiles: 43
Smiles: 44
Smiles: 45
Smiles: 46
Smiles: 47
Smiles: 48
Smiles: 49
Smiles: 50
Smiles: 51
Smiles: 52
Smiles: 53
Smiles: 54
Smiles: 55
Smiles: 56
Smiles: 57
Smiles: 58
Smiles: 59
Smiles: 60
Smiles: 61
Smiles: 62
Smiles: 63
Smiles: 64
Smiles: 65
Smiles: 66
Smiles: 67
Smiles: 68
Smiles: 69
Smiles: 70
Smiles: 71
Smiles: 72
Smiles: 73
Smiles: 74
Smiles: 75
Smiles: 76
Smiles: 77
Smiles: 78
Smiles: 79
Smiles: 80
Smiles: 81
Smiles: 82
Smiles: 83
Smiles: 84
Smiles: 85
Smiles: 86
Smiles: 87
Smiles: 88
Smiles: 89
Smiles: 90
Smiles: 91
Sm

In [7]:
data = {'fingerprint': fingerprint, 'pred_out': pred_output, 'orig_out': original_output }
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)
df

,fingerprint,pred_out,orig_out
0,"[0.038528792560100555, -0.060478050261735916, ...",[-1.6442649364471436],0.4343
1,"[-0.03517939895391464, -0.2776426076889038, -1...",[-1.5285539627075195],0.8740
2,"[0.11787766963243484, -0.558820903301239, -1.6...",[-1.4760966300964355],1.1415
3,"[-0.0772891715168953, -1.3392486572265625, -1....",[-1.0364990234375],1.5240
4,"[1.1507545709609985, -1.1616188287734985, -0.6...",[-1.937893033027649],0.4489
...,...,...,...
363,"[0.8615942597389221, -0.9150235652923584, 0.71...",[-0.4574372470378876],1.5387
364,"[2.93359112739563, -0.7651606202125549, 0.9211...",[-0.8315487504005432],1.3385
365,"[-0.3269875943660736, -0.46392083168029785, 0....",[1.5946460962295532],3.8901
366,"[1.1184444427490234, -0.38528648018836975, 0.6...",[0.5692479610443115],2.7568


In [8]:
df = pd.read_csv('data.csv')
df

,fingerprint,pred_out,orig_out
0,"[0.038528792560100555, -0.060478050261735916, ...",[-1.6442649364471436],0.4343
1,"[-0.03517939895391464, -0.2776426076889038, -1...",[-1.5285539627075195],0.8740
2,"[0.11787766963243484, -0.558820903301239, -1.6...",[-1.4760966300964355],1.1415
3,"[-0.0772891715168953, -1.3392486572265625, -1....",[-1.0364990234375],1.5240
4,"[1.1507545709609985, -1.1616188287734985, -0.6...",[-1.937893033027649],0.4489
...,...,...,...
363,"[0.8615942597389221, -0.9150235652923584, 0.71...",[-0.4574372470378876],1.5387
364,"[2.93359112739563, -0.7651606202125549, 0.9211...",[-0.8315487504005432],1.3385
365,"[-0.3269875943660736, -0.46392083168029785, 0....",[1.5946460962295532],3.8901
366,"[1.1184444427490234, -0.38528648018836975, 0.6...",[0.5692479610443115],2.7568
